In [1]:
!pip install xgboost==1.5.2

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com


In [2]:
import pandas as pd

In [3]:
X_train = pd.read_parquet('X_train.parquet')

In [4]:
X_test = pd.read_parquet('X_test.parquet')

In [5]:
y_train = X_train['label']
y_test = X_test['label']
X_train = X_train[X_train.columns.difference(['label'])]
X_test = X_test[X_test.columns.difference(['label'])]

In [6]:
import xgboost as xgb

In [7]:
xgb.__version__

'1.5.2'

In [8]:
dtrain = xgb.DMatrix(
        X_train,
        y_train )

In [9]:
dtest = xgb.DMatrix(
        X_test,
        y_test    )

In [10]:
xgb_params = {
    'max_depth':         8,
    'tree_method':       'gpu_hist',
    'objective':         'binary:logistic',
    'eval_metric':       'aucpr',
    'predictor':         'gpu_predictor',
}

In [11]:
n_rounds = 2000

In [12]:
import time

In [13]:
start = time.time()
model = xgb.train(params=xgb_params, 
                       dtrain=dtrain, 
                       num_boost_round=n_rounds)
print("Training Time", time.time()-start, "seconds")

Training Time 129.1342523097992 seconds


In [14]:
# aucpr baseline
y_test.sum()/len(y_test) 

0.0012174113515407406

In [19]:
y_score = model.predict(dtest)

In [20]:
from sklearn.metrics import average_precision_score

y_true = y_test.values
ap = average_precision_score(y_true, y_score)
print(f'Test Area Under Precision Recall Curve: {ap: 0.4f}')

Test Area Under Precision Recall Curve:  0.8685


In [21]:
model_path = "./xgboost.json"

In [22]:
model.save_model(model_path)

In [23]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}